In [2]:
import scores.categorical
import xarray as xr

## Overview of Binary Contingency Scores

Many people will be familiar with the following terms: true positives, false negatives, true positive rate. These terms and many more all apply to a binary contingency table. Producing these categorical statistics is inherently a three-step process.

1. For data that is either continuous or categorical, first convert it to binary data using an "event definition". For data that is already binary, go straight to step two.
2. Produce a contingency manager object, which:
   - stores the true positives, true negativates, false positives and false negatives, and
   - allows the data to be aggragated and transformed (for example, reducing the latitude and longitude dimensions while preserving the height dimension in order to assess the accuracy at different vertical levels).
3. Calculate the desired metrics from the contingency tables produced in step two.

`scores` provides support for all three steps, plus convenient functions for working efficiently. Most of the `scores` APIs are functional in nature, however introducing some classes (see below) will make contingency metrics much easier to calculate and work with. This notebook starts with simple examples and builds up to more complex ones.

## Step One: Converting continuous (or categorical) data into binary data

In some instances, data will already be binary. Examples of binary data include forecasts of "events" - such as "it will rain" or "a tropical cyclone will occur". 

**If the data is already binary, ignore this step and go straight to step two.**

However, in many situations, data is not binary and is instead either continuous or categorical. An example of continuous data is a physical quantity, such as a temperature. An example of categorical data is the [Beaufort Wind Scale](https://www.rmets.org/metmatters/beaufort-wind-scale).

In order to use binary contingency scores, any non-binary data must first be converted into binary data. 

Options include either:  
- convert the non-binary data into binary data outside of `scores`, then input the binary data into `scores` (in which case, go to step two), OR  
- use `scores` to efficiently convert the non-binary data into binary data, using the `scores` "event operator" class. 

`scores` provides a general class called an "EventOperator". `scores` provides in-built event operator types (e.g. greater-than, less-than or many others). Alternatively, users can define their own event operator implementation for use within `scores`.
    
The rest of this section focuses on using `scores` to convert continuous data into binary data using the "event operator" class.

For instance, let's say you need to verify how accurately a numerical weather prediction (NWP) system forecasts "heavy rainfall". As the NWP preciptation rate predictions are continuous, you will need to first convert the precitation rate predictions into "heavy rainfall events" as binary data. To do this, you:  
- first define an "event operator" - in this case by defining what constitutes a "heavy rainfall event", for example precitation above a specified threshold, and  
- then apply this "heavy rainfall event" operator to the continuous precipiation rate prediction data - this will convert the predictions into binary data - into either "heavy rainfall event = true" or "heavy rainfall event = false".

A more complex event like "a thunderstorm" may involve more complex logic to determine when the event occurs, incorporating factors such as wind and lightning data as well as the rainfall rate. 

### Worked Example

Here is an example of using `scores` to convert non-binary data into binary data using the "event operator" class.

Consider the following two tables of forecast and observed values for precipitation.

In [3]:
# Provides a basic forecast data structure in three dimensions
simple_forecast = xr.DataArray(
    [
		[
			[0.9, 0.0,   5], 
			[0.7, 1.4, 2.8],
			[.4,  0.5, 2.3],
		], 
			[
			[1.9, 1.0,  1.5], 
			[1.7, 2.4,  1.1],
			[1.4,  1.5, 3.3],
		], 
	],
	coords=[[10, 20], [0, 1, 2], [5, 6, 7]], dims=["height", "lat", "lon"])

In [4]:
# Within 0.1 or 0.2 of the forecast in all cases except one
# Can be used to find some exact matches, and some close matches
simple_obs = xr.DataArray(
    [
		[
			[0.9, 0.0,   5], 
			[0.7, 1.3, 2.7],
			[.3,  0.4, 2.2],
		], 
			[
			[1.7, 1.2,  1.7], 
			[1.7, 2.2,  3.9],
			[1.6,  1.2, 9.9],
		], 
	],
	coords=[[10, 20], [0, 1, 2], [5, 6, 7]], dims=["height", "lat", "lon"])

For the purposes of this example, let's say an event occurs if the precipitation value is greater than 1.3. We therefore define a threshold based event operator with a threshold of 1.3.

Applying this operator, means `scores` converts the non-binary data into binary data. It does this by creating binary event data - precipitation data with a value less than or equal to 1.3 is "false" and precipitation data with a value greater than 1.3 is "true".

The following code creates a "greater than" operator  -  using `scores` built-in ThresholdEventOperator  -  which creates events with a threshold of "> 1.3". It can be repeatedly called with different event thresholds, for reasons that will be explained later.


In [12]:
# An event here is defined as precipitation with a value greater than 1.3

# The EventThresholdOperator can take a variety of operators from the python "operator" module, or a user-defined function
# The default is operator.gt, which is the same as ">" but in functional form.

event_operator = scores.categorical.ThresholdEventOperator(default_event_threshold=1.3)

In [13]:
# The event operator creates binary "event tables". 
# If you don't wish to visualise the binary "event tables", you can ignore this section.
# However, if you wish to visualise or utilise the the "event tables" directly, you can do so as follows:

forecast_binary, observed_binary = event_operator.make_event_tables(simple_forecast, simple_obs)
print(forecast_binary)

<xarray.DataArray (height: 2, lat: 3, lon: 3)> Size: 18B
array([[[False, False,  True],
        [False,  True,  True],
        [False, False,  True]],

       [[ True, False,  True],
        [ True,  True, False],
        [ True,  True,  True]]])
Coordinates:
  * height   (height) int64 16B 10 20
  * lat      (lat) int64 24B 0 1 2
  * lon      (lon) int64 24B 5 6 7


## Step Two: Making Binary Contingency Information

Once the event tables have been made, the next step is to make a contingency manager object. It is possible (and quicker) to make a contingency manager from the event operator directly, but we will first show the process as separate steps, and then show how to do the two things at once. The contingency manager stores the binary forecast event data, the binary observed event data and the contingency table itself, plus provides functions for aggregating the data along various dimensions, and calculating metrics from that information.

In [10]:
contingency_manager = scores.categorical.BinaryContingencyManager(forecast_binary, observed_binary)
print(contingency_manager)  # Print a view of the contingency table

Contingency Manager (xarray view of table):
<xarray.DataArray (contingency: 5)> Size: 40B
array([ 9,  6,  2,  1, 18])
Coordinates:
  * contingency  (contingency) <U11 220B 'tp_count' 'tn_count' ... 'total_count'


In [12]:
# Uncomment this line to view the full functionality provided by a contingency manager
# help(contingency_manager)

In [14]:
# It is also possible, and briefer, to make the binary contingency table directly from the event operator, as follows:

contingency_manager = event_operator.make_table(simple_forecast, simple_obs)
print(contingency_manager)

Contingency Manager (xarray view of table):
<xarray.DataArray (contingency: 5)> Size: 40B
array([ 9,  6,  2,  1, 18])
Coordinates:
  * contingency  (contingency) <U11 220B 'tp_count' 'tn_count' ... 'total_count'


In [15]:
# It is also possible to retrieve the contingency table as an xarray object

actual_table = contingency_manager.get_table()
actual_table

<xarray.DataArray (contingency: 5)> Size: 40B
array([ 9,  6,  2,  1, 18])
Coordinates:
  * contingency  (contingency) <U11 220B 'tp_count' 'tn_count' ... 'total_count'

In [16]:
# Further, if it turn out you want them, the event tables are still there 

contingency_manager.forecast_events

<xarray.DataArray (height: 2, lat: 3, lon: 3)> Size: 18B
array([[[False, False,  True],
        [False,  True,  True],
        [False, False,  True]],

       [[ True, False,  True],
        [ True,  True, False],
        [ True,  True,  True]]])
Coordinates:
  * height   (height) int64 16B 10 20
  * lat      (lat) int64 24B 0 1 2
  * lon      (lon) int64 24B 5 6 7

## Step Three: Calculating Binary Contingency Scores

The binary contingency manager can then be utilised to calculate a wide variety of scores which are based on the contingency table.

Below are two examples of using `scores` to calculate metrics directly from the contigency manager:
- "accuracy": (true positives + false negatives)/(total number of samples)
- "false_alarm_rate": (number of false positives)/(total number of samples)

In [17]:
contingency_manager.accuracy()

<xarray.DataArray ()> Size: 8B
array(0.83333333)

In [18]:
contingency_manager.false_alarm_rate()

<xarray.DataArray ()> Size: 8B
array(0.25)

## Tranforming and Computing Large Tables 

Using the "transform" operation on the contingency table can be helpful for handling very large data sets.

Transforming the table triggers the 'compute' function against the dask arrays which until this point may have been loaded lazily. This is insignificant for small data sets, but if calculating a contingency table for a multi-terabyte gridded data set, can be time-consuming. The tranformed table contains only the resultant contingency table, which is thereafter very fast to work with.

Here we present a 'transform' with no arguments to trigger the computation of the aggregated contingency table. As such, it may be an expensive operation, but the resultant table is very small and all the relevant scores can be easily calculated as the table is re-used.

In [19]:
computed = contingency_manager.transform()
computed.accuracy()

<xarray.DataArray ()> Size: 8B
array(0.83333333)

## Exploring Dimensionality

The transform function can also be utilised to explore the dimensionality of the data. This function accepts preserve_dims and reduce_dims as per the rest of the `scores` codebase. 

While the team behind `scores` intend to add functionality so that `scores` can support weights during transformation, this functionality has not yet been added.

At present, the following approach can be used to examine contingency scores along the data dimensions.

In [20]:
# If it is wanted, the underlying event counts can be accessed
new_manager = contingency_manager.transform(preserve_dims='height')
print(new_manager)

Contingency Manager (xarray view of table):
<xarray.DataArray (contingency: 5, height: 2)> Size: 80B
array([[3, 6],
       [5, 1],
       [1, 1],
       [0, 1],
       [9, 9]])
Coordinates:
  * height       (height) int64 16B 10 20
  * contingency  (contingency) <U11 220B 'tp_count' 'tn_count' ... 'total_count'


In [22]:
# Calculating the accuracy on this transformed manager will show the accuracy along the height dimension

new_manager.accuracy()

<xarray.DataArray (height: 2)> Size: 16B
array([0.88888889, 0.77777778])
Coordinates:
  * height   (height) int64 16B 10 20

## Next Steps

Some interesting next steps could include:
 - Calculating multiple contingency tables for different thresholds to observe how accuracy changes according to different event thresholds